In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging 

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import json
import polars as pl
from polars import col, lit
import altair as alt
import simrs
from history import History, tabular
import input_builder as b
import utils 

In [3]:
# Agents
agent_ids = ["a" +str(i) for i in range(1, 20)] 

# Goods
wheat = "Wheat"
goods = [wheat]

# Ports
genoa = "Genoa"
rome = "Rome"
milan = "Milan"
venice = "Venice"
marsailles = "Marsailles"
port_ids = [genoa, rome, milan, venice, marsailles]

# genoa -> milan -> marsailles
#   v         v
# rome -> venice
edges = [(genoa, milan), (milan, marsailles), (rome, venice), (rome, genoa), (venice, milan)]
_market = lambda net: b._market(wheat, b._market_info(net=net)) 

# net balanced
ports = [
  b._port(genoa, _market(2)), 
  b._port(milan, _market(1)), 
  b._port(rome, _market(0)), 
  b._port(venice, _market(-1)),
  b._port(marsailles, _market(-2)), 
]

_agent = lambda id, pos: b._agent(id, pos, 1000, "Greedy")
agents = [_agent(id, port_id) for (id, port_id) in zip(agent_ids, port_ids * 20)] 

x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=100))
history = simrs.run(x)
(actions, agents, markets, events) = tabular(history)
markets = markets.select(pl.exclude("pricer"))

In [4]:
print("agents") 
agents.head(2)

agents


behavior,cargo,coins,id,pos,tick
str,str,f64,str,str,i64
"""Greedy""",null,1000.0,"""a3""","""Milan""",0
"""Greedy""",null,1000.0,"""a11""","""Genoa""",0


In [5]:
print("markets") 
markets.head(2)

markets


consumption,good,port,price,production,supply,tick
f64,str,str,f64,f64,f64,i64
100.0,"""Wheat""","""Marsailles""",100.0,98.0,1000.0,0
100.0,"""Wheat""","""Milan""",100.0,101.0,1000.0,0


In [6]:
print("actions")
actions.head(10)

actions


action,agent_id,tick,good,port_id
str,str,i64,str,str
"""Noop""","""a3""",0,null,null
"""Noop""","""a11""",0,null,null
"""Noop""","""a4""",0,null,null
"""Noop""","""a6""",0,null,null
"""Noop""","""a10""",0,null,null
"""Noop""","""a12""",0,null,null
"""Noop""","""a15""",0,null,null
"""Noop""","""a9""",0,null,null
"""Noop""","""a17""",0,null,null


In [7]:
events.head(10)

agent,amt,cost,event,good,port,tick
str,i64,f64,str,str,str,i64
"""a3""",1,99.9,"""Trade""","""Wheat""","""Milan""",1
"""a11""",1,99.7,"""Trade""","""Wheat""","""Genoa""",1
"""a6""",1,99.9,"""Trade""","""Wheat""","""Genoa""",1
"""a12""",1,100.1,"""Trade""","""Wheat""","""Rome""",1
"""a17""",1,100.3,"""Trade""","""Wheat""","""Rome""",1
"""a16""",1,100.1,"""Trade""","""Wheat""","""Genoa""",1
"""a8""",1,100.1,"""Trade""","""Wheat""","""Milan""",1
"""a13""",1,100.3,"""Trade""","""Wheat""","""Milan""",1
"""a7""",1,100.5,"""Trade""","""Wheat""","""Rome""",1


In [8]:
def plot_agents(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y=alt.Y('coins:Q').scale(zero=False),
        color=alt.Color('id:O').scale(scheme='dark2'),
    )
    lines = base.transform_loess('tick', 'coins', bandwidth=.5, groupby=['id']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
  
plot_agents(agents)

alt.LayerChart(...)

In [9]:
def plot_agent_locations(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y='pos:N',
        color=alt.Color('id:O').scale(scheme='dark2'),
    )
    return base.mark_point().interactive()
plot_agent_locations(agents)

alt.Chart(...)

In [10]:
def plot_prices_by_port(ports: pl.DataFrame, color='dark2'):
    base = alt.Chart(ports).encode(
        x='tick:Q',
        y='price:Q',
        color=alt.Color('port:O').scale(scheme=color),
    )
    lines = base.transform_loess('tick', 'price', bandwidth=.5, groupby=['port']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
plot_prices_by_port(markets)

alt.LayerChart(...)

In [11]:
def make_routes(events):
    trade_events = events.filter(events["event"] == "Trade")
    def foo(df):
        buys = df.filter(df["amt"] > 0).select(
            'agent', 
            'amt', 
            pl.col("cost").alias('buy_cost'), 
            pl.col('port').alias('src'), 
            pl.col('tick').alias('buy_tick')
        )
        sells = df.filter(df["amt"] < 0).select(
            pl.col('cost').alias('sell_cost'), 
            pl.col('port').alias('dst'),
            pl.col('tick').alias('sell_tick')
        )

        df = pl.concat( [ buys, sells ], how='horizontal')
        df = df.with_columns((-df["sell_cost"] - df["buy_cost"]).alias("profit"))
        df = df.with_columns((df['buy_cost'] / df['amt']).alias('buy_price'))
        df = df.with_columns((df['sell_cost'] / -df['amt']).alias('sell_price'))
        return df

    return trade_events.groupby('agent').apply(foo)
routes = make_routes(events)
routes.head()

agent,amt,buy_cost,src,buy_tick,sell_cost,dst,sell_tick,profit,buy_price,sell_price
str,i64,f64,str,i64,f64,str,i64,f64,f64,f64
"""a7""",1,100.5,"""Rome""",1,-99.9,"""Venice""",2,-0.6,100.5,99.9
"""a7""",1,100.3,"""Venice""",3,-100.3,"""Milan""",4,0.0,100.3,100.3
"""a7""",1,99.1,"""Milan""",5,-101.3,"""Marsailles""",6,2.2,99.1,101.3
"""a7""",1,101.7,"""Milan""",8,-100.7,"""Venice""",9,-1.0,101.7,100.7
"""a7""",1,102.7,"""Milan""",11,-101.5,"""Marsailles""",12,-1.2,102.7,101.5


In [12]:
def plot_trades(trades: pl.DataFrame):
    base = alt.Chart(trades).encode(
        x='sell_tick:Q',
        y='profit:Q',
        color=alt.Color('agent_id:N').scale(scheme='dark2'),
    )
    return base.mark_point().interactive()
plot_trades(routes)

alt.Chart(...)

In [13]:
def plot_buy_and_sell_prices(trades: pl.DataFrame):
    buy = alt.Chart(trades).encode(
        x='buy_tick:Q',
        y='buy_price:Q',
        color=alt.Color('dst:N').scale(scheme='dark2'),
    )
    sell = alt.Chart(trades).encode(
        x='sell_tick:Q',
        y='sell_price:Q',
        color=alt.Color('dst:N').scale(scheme='dark2'),
    )
    
    return (buy.mark_point()+sell.mark_point()).interactive()

plot_buy_and_sell_prices(routes) + plot_trades(routes)

alt.LayerChart(...)

In [14]:
## Where would prices have been if agents didn't trade?
# x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=100))
def no_agent_markets(input_format) -> pl.DataFrame:
    input_format['agents'] = []
    no_agent_history = simrs.run(input_format)
    (_, _, no_agent_markets,_) = tabular(no_agent_history)
    return no_agent_markets.select(pl.exclude("pricer"))

plot_prices_by_port(no_agent_markets(x)) 

alt.LayerChart(...)

In [15]:
plot_prices_by_port(markets)

alt.LayerChart(...)

## Port level analysis
- How much does each port trade?
- Trade volume bucketted 
- Volume in dollars 
- Biggest trading partners (other ports)

In [16]:
## Total goods traded per port
events.groupby("port").agg(pl.sum("amt"))
# def plot_

port,amt
str,i64
"""Marsailles""",-114
"""Venice""",-62
"""Rome""",35
"""Milan""",114
"""Genoa""",27


Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)